<a href="https://colab.research.google.com/github/dolDolSee/TF2.0/blob/main/CIFAR10_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
from tensorflow.keras.applications import VGG16, ResNet50, ResNet50V2, Xception

In [ ]:
model = VGG16()
model.summary()

In [6]:
model = VGG16(input_shape=(32,32,3), include_top=False, weights='imagenet')
model.summary()

58900480/58889256 [==============================] - 2s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                              

In [7]:
print(model)
print(model.output)

KerasTensor(type_spec=TensorSpec(shape=(None, 1, 1, 512), dtype=tf.float32, name=None), name='block5_pool/MaxPool:0', description="created by layer 'block5_pool'")


In [37]:
IMAGE_SIZE=32
BATCH_SIZE=64

In [35]:
from tensorflow.keras.datasets import fashion_mnist
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import pandas as pd
import random as python_random
from tensorflow.keras.datasets import cifar10

(train_images, train_labels),(test_images, test_labels) = fashion_mnist.load_data()

# 0 ~ 1사이값의 float32로 변경하는 함수
def get_preprocessed_data(images, labels, scaling=True):
    
    # 학습과 테스트 이미지 array를 0~1 사이값으로 scale 및 float32 형 변형. 
    if scaling:
        images = np.array(images/255.0, dtype=np.float32)
    else:
        images = np.array(images, dtype=np.float32)
        
    labels = np.array(labels, dtype=np.float32)
    
    return images, labels

def get_preprocessed_ohe(images, labels):
    images, labels = get_preprocessed_data(images, labels, scaling=False)
    # OHE 적용 
    oh_labels = to_categorical(labels)
    return images, oh_labels

# 학습/검증/테스트 데이터 세트에 전처리 및 OHE 적용한 뒤 반환 
def get_train_valid_test_set(train_images, train_labels, test_images, test_labels, valid_size=0.15, random_state=2021):
    # 학습 및 테스트 데이터 세트를  0 ~ 1사이값 float32로 변경 및 OHE 적용. 
    train_images, train_oh_labels = get_preprocessed_ohe(train_images, train_labels)
    test_images, test_oh_labels = get_preprocessed_ohe(test_images, test_labels)
    
    # 학습 데이터를 검증 데이터 세트로 다시 분리
    tr_images, val_images, tr_oh_labels, val_oh_labels = train_test_split(train_images, train_oh_labels, test_size=valid_size, random_state=random_state)
    
    return (tr_images, tr_oh_labels), (val_images, val_oh_labels), (test_images, test_oh_labels ) 

# CIFAR10 데이터 재 로딩 및 Scaling/OHE 전처리 적용하여 학습/검증/데이터 세트 생성. 
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)

(tr_images, tr_oh_labels), (val_images, val_oh_labels), (test_images, test_oh_labels) = \
    get_train_valid_test_set(train_images, train_labels, test_images, test_labels, valid_size=0.15, random_state=2021)

print(tr_images.shape, tr_oh_labels.shape, val_images.shape, val_oh_labels.shape, test_images.shape, test_oh_labels.shape)

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)
(42500, 32, 32, 3) (42500, 10) (7500, 32, 32, 3) (7500, 10) (10000, 32, 32, 3) (10000, 10)


In [16]:
print(tr_images.shape)

(51000, 28, 28)


In [32]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import Accuracy

class VGGclass(tf.keras.Model):
  def __init__(self):
    super(VGGclass, self).__init__()

    self.VGG16 = tf.keras.applications.VGG16(input_shape=(32,32,3), include_top=False, weights="imagenet")
    # VGG16_output = self.VGG16.output
    self.pooling = tf.keras.layers.GlobalAveragePooling2D()
    self.Dense = tf.keras.layers.Dense(50, activation='relu')
    self.outputlayer = tf.keras.layers.Dense(10, activation="softmax")

  def call(self,x):
    input = tf.reshape(x,[-1,32,32,3])
    x = self.VGG16(input)
    x = self.pooling(x)
    x = self.Dense(x)
    x = self.outputlayer(x)
    return x

model = VGGclass()
model.compile(optimizer = Adam(0.001),loss='categorical_crossentropy',metrics=['Accuracy'])
history = model.fit(flow_tr_gen, epochs=40, validation_data = flow_val_gen)

Epoch 1/40
665/665 [==============================] - 58s 83ms/step - loss: 1.8008 - Accuracy: 0.2825 - val_loss: 1.3731 - val_Accuracy: 0.4632
Epoch 2/40
665/665 [==============================] - 54s 81ms/step - loss: 1.2336 - Accuracy: 0.5327 - val_loss: 1.0506 - val_Accuracy: 0.6333
Epoch 3/40
665/665 [==============================] - 54s 81ms/step - loss: 0.9734 - Accuracy: 0.6613 - val_loss: 0.8749 - val_Accuracy: 0.6995
Epoch 4/40
665/665 [==============================] - 54s 81ms/step - loss: 0.8004 - Accuracy: 0.7330 - val_loss: 0.7635 - val_Accuracy: 0.7449
Epoch 5/40
665/665 [==============================] - 54s 81ms/step - loss: 0.7015 - Accuracy: 0.7680 - val_loss: 0.6746 - val_Accuracy: 0.7775
Epoch 6/40
665/665 [==============================] - 54s 81ms/step - loss: 0.6249 - Accuracy: 0.7938 - val_loss: 0.6582 - val_Accuracy: 0.7897
Epoch 7/40
665/665 [==============================] - 54s 81ms/step - loss: 0.5578 - Accuracy: 0.8179 - val_loss: 0.6439 - val_Accuracy:

In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(
    horizontal_flip=True,
    rescale=1/255.0
)
valid_generator = ImageDataGenerator(rescale=1/255.0)

flow_tr_gen = train_generator.flow(tr_images, tr_oh_labels, batch_size=BATCH_SIZE, shuffle=True)
flow_val_gen = valid_generator.flow(val_images, val_oh_labels, batch_size=BATCH_SIZE, shuffle=False)

In [39]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
test_generator = ImageDataGenerator(rescale=1/255.0)
flow_test_gen = test_generator.flow(test_images, test_oh_labels, batch_size=BATCH_SIZE, shuffle=False)
model.evaluate(flow_test_gen)

157/157 [==============================] - 4s 28ms/step - loss: 0.7662 - Accuracy: 0.8350


[0.7662031650543213, 0.8349999785423279]

In [40]:
model.summary()

Model: "vg_gclass_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 global_average_pooling2d_5   multiple                 0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_10 (Dense)            multiple                  25650     
                                                                 
 dense_11 (Dense)            multiple                  510       
                                                                 
Total params: 14,740,848
Trainable params: 14,740,848
Non-trainable params: 0
_________________________________________________________________
